In [26]:
import os
import numpy as np
import shutil
import open3d as o3d

In [27]:
def read_pc(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    points = []
    for line in lines[11:]:
        values = [float(v) for v in line.strip().split()]
        points.append(values)
    points = np.asarray(points)

    return points

def get_pose(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    poses = []
    for line in lines:
        values = [float(v) for v in line.strip().split()]
        poses.append(values)

    return poses

In [28]:
seqs = ['scenario2', 'scenario3', 'scenario5', 'scenario6', 'scenario8']
root_path = "/ws/data/erasor_carla/carla_dataset/"
debug_path = "/ws/data/erasor_carla/carla_dataset/debug/"
seq = "scenario2"


In [29]:
# ### change the file name of the carla_dataset ###
# for seq in seqs:
#     file_dir = os.path.join(root_path, "testing_data", seq, "local/scan/")
#     filename_list = os.listdir(file_dir)
#     for filename in filename_list:
#         orig_path = os.path.join(file_dir, filename)
#         print(orig_path)
#         new_filename = str(filename[:-4].zfill(6))+".pcd"
#         new_path = os.path.join(file_dir, new_filename)
#         print(new_path)
#         os.rename(orig_path, new_path)
    


In [49]:
seq = "scenario2"
scan_dir = os.path.join(root_path, "testing_data", seq, "global/scan/")
odom_path = os.path.join(root_path, "testing_data", seq, "odom/scan/odometry.txt")
map_path = os.path.join(root_path, "testing_map/v0.1", seq, "map.pcd")
odoms = get_pose(odom_path)

scan_filenames = sorted(os.listdir(scan_dir))
scan_files = [os.path.join(scan_dir, x) for x in scan_filenames]
# print(scan_filenames)

for idx, scan_filename in enumerate(scan_filenames[20:22]):
    scan_file = os.path.join(scan_dir, scan_filename)
    scan_dst = os.path.join(debug_path, scan_filename)
    map_dst = os.path.join(debug_path, "map_" + scan_filename)
    print(map_dst)
    shutil.copy(scan_file, scan_dst)
    
    odom = odoms[idx]
    radius = 50
#     map_pcd = o3d.io.read_point_cloud(map_path)
#     map_points = np.asarray(map_pcd.points)0
    map_data = read_pc(map_path)
    map_points, map_labels = map_data[:, :3], map_data[:, 3]
    map_points = map_points[np.sum(np.square(map_points[:, :3] - odom), axis=-1) < radius*radius]
    label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}
    map_colors = np.array([label2color[x] for x in map_labels])
    print("map_data shape: ", map_data.shape)
    print("map_points shape: ", map_points.shape)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(map_points)
    pcd.colors = o3d.utility.Vector3dVector(map_colors)
    o3d.io.write_point_cloud(map_dst, pcd)
    
    scan_data = read_pc(scan_file)
    scan_points, scan_labels = scan_data[:, :3], scan_data[:, 3]
    print("scan_data shape: ", scan_data.shape)
    print("scan_points shape: ", scan_points.shape)


/ws/data/erasor_carla/carla_dataset/debug/map_000020.pcd
map_data shape:  (3597868, 4)
map_points shape:  (2745469, 3)
scan_data shape:  (126206, 4)
scan_points shape:  (126206, 3)
/ws/data/erasor_carla/carla_dataset/debug/map_000021.pcd
map_data shape:  (3597868, 4)
map_points shape:  (2745501, 3)
scan_data shape:  (126185, 4)
scan_points shape:  (126185, 3)


In [43]:
label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}

In [44]:
colors = np.array([label2color[x] for x in map_labels])

In [ ]:
# # save scan npy data
# for seq in seqs:
#     scan_dir = os.path.join(root_path, "testing_data", seq, "global/scan/")
# #     new_scan_dir = os.path.join(root_path, "testing_data", seq, "global_npz")
#     odom_path = os.path.join(root_path, "testing_data", seq, "odom/scan/odometry.txt")
#     odoms = get_pose(odom_path)
# #     if not os.path.exists(new_scan_dir):
# #         os.makedirs(new_scan_dir)
#         # os.makedirs(os.path.join(new_scan_dir, "scan"))
#         # new_scan_dir = os.path.join(new_scan_dir, "scan")
#     scan_filenames = os.listdir(scan_dir)
#     scan_files = [os.path.join(scan_dir, x) for x in scan_filenames]
#     dummy = scan_files
#     for idx, scan_filename in enumerate(scan_filenames):
#         scan_file = os.path.join(scan_dir, scan_filename)
#         dst = os.apth.join(debug_path, scan_filename)
#         shutil.copy(scan_file, dst)
#         points = read_pc(scan_file)
#         odom = odoms[idx]
#         new_scan_file = os.path.join(new_scan_dir, scan_filename[:-4].zfill(6))
#         np.savez(new_scan_file, points, odom)
#         # np.save(new_scan_file, points)